# LSMS Foreward Kinematics

Create a scene with CSYS

In [1]:
import bumblebee as bb
import numpy as np

In [2]:
assm = bb.CSYS()
def home_view():
    assm.ax.set_xlim([-125, 125])
    assm.ax.set_ylim([-250, 0])
    assm.ax.set_zlim([0,250])
home_view()

Add the LSMS and set the link nodes to variable names

In [3]:
LSMS = bb.RigidCollection.from_json('LSMS.json')
assm.add(LSMS)

In [4]:
kingpost=None
arm_assm=None
forearm_assm=None
slide_assm = None

for node in LSMS.nodes:
    kingpost = node if node.name == 'KingpostAsm' else kingpost
    arm_assm = node if node.name == 'ArmAsm' else arm_assm
    forearm_assm = node if node.name == 'ForearmAsm' else forearm_assm
    slide_assm = node if node.name == 'SlideAssm' else slide_assm

forearm_hoist = None
arm_hoist = None
king_hoist = None

for node in kingpost.nodes:
    forearm_hoist = node if node.name == 'ForearmHoist' else forearm_hoist
    king_hoist = node if node.name == 'KingHoist' else king_hoist    
    arm_hoist = node if node.name == 'ArmHoist' else arm_hoist
    
arm = None
crown = None
wing = None
tail = None
    
for node in arm_assm.nodes:
    arm = node if node.name == 'ArmAsm' else arm
    crown = node if node.name == 'CrownAssm' else crown
    wing = node if node.name == 'WingAssm' else wing
    tail = node if node.name == 'TailAssm' else tail
    
forearm = None
antenna = None
whisker = None

for node in forearm_assm.nodes:
    forearm = node if node.name == 'ForearmAssm' else forearm
    whisker = node if node.name == 'WhiskerAssm' else whisker
    antenna = node if node.name == 'AntennaAssm' else antenna
    
gantry = None    
stewart = None    
for node in slide_assm.nodes:
    gantry = node if node.name == 'Gantry' else gantry
    stewart = node if node.name == 'StewartAssm' else stewart
    
bplate = None
tplate = None
for node in stewart.nodes:
    bplate = node if node.name == 'BottomPlateAssm' else bplate
    tplate = node if node.name == 'TopPlateAssm' else tplate

Get the joint positions and link orientations at the home position

In [5]:
Mlist = []
plist = []

for link in [kingpost, arm, forearm]:
    for node in link.nodes:
        if node.name == 'M_pose':
            Mlist.append(node.tf.copy())
        elif node.name == 'Joint':
            plist.append(node.tf[0:3,3].copy())
            
Mlist.append(slide_assm.tf.copy())

ArmMs = []
for link in [arm, crown, wing, tail]:
    for node in link.nodes:
        if node.name.endswith('M'):
            ArmMs.append(node)
            
ArmMs.append(arm_hoist)
            
ForearmMs = []
for link in [forearm, antenna, whisker]:
    for node in link.nodes:
        if node.name.endswith('M'):
            ForearmMs.append(node)
            
ForearmMs.append(forearm_hoist)
            
collision = None
whisk_joint = None
whisk_M = None
for node in whisker.nodes:
    collision = node if node.name == 'Collision' else collision
    whisk_joint = node if node.name == 'WhiskerJoint' else whisk_joint
    whisk_M = node if node.name == 'WhiskerM' else whisk_M
    
ant_joint = None
ant_M = None
for node in antenna.nodes:
    ant_joint = node if node.name == 'AntennaJoint' else ant_joint
    ant_M = node if node.name == 'AntennaM' else ant_M
    
wire_pt = None
for node in arm.nodes:
    wire_pt = node if node.name == 'WirePt' else wire_pt

Create a list of screw axes from these positions

In [6]:
import modern_robotics as mr

w_king = [0,0,1] #rotates about z axis
p_king = plist[0]
S_king = mr.ScrewToAxis(p_king, w_king, 0)

w_arm = [1,0,0] #rotates about x axis
p_arm = plist[1]
S_arm = mr.ScrewToAxis(p_arm, w_arm, 0)

w_forearm = [1,0,0] #rotates about x axis
p_forearm = plist[2]
S_forearm = mr.ScrewToAxis(p_forearm, w_forearm, 0)

# translate along y axis
S_slider = [0,0,0,0,-1,0]

Slist = [S_king, S_arm, S_forearm, S_slider]

links = [kingpost, arm_assm, forearm_assm, slide_assm]

for link, M in zip(links, Mlist):
    link.relpose = mr.TransInv(M) @ link.tf

In [7]:
wires, = assm.ax.plot(
    [Mlist[1][0,3], ArmMs[0].tf[0,3], ArmMs[1].tf[0,3], ArmMs[2].tf[0,3], ArmMs[3].tf[0,3]],
    [Mlist[1][1,3], ArmMs[0].tf[1,3], ArmMs[1].tf[1,3], ArmMs[2].tf[1,3], ArmMs[3].tf[1,3]],
    [Mlist[1][2,3], ArmMs[0].tf[2,3], ArmMs[1].tf[2,3], ArmMs[2].tf[2,3], ArmMs[3].tf[2,3]],
    'k'
)

wires2, = assm.ax.plot(
    [Mlist[2][0,3], ForearmMs[0].tf[0,3], ForearmMs[1].tf[0,3], wire_pt.tf[0,3], ForearmMs[2].tf[0,3]],
    [Mlist[2][1,3], ForearmMs[0].tf[1,3], ForearmMs[1].tf[1,3], wire_pt.tf[1,3], ForearmMs[2].tf[1,3]],
    [Mlist[2][2,3], ForearmMs[0].tf[2,3], ForearmMs[1].tf[2,3], wire_pt.tf[2,3], ForearmMs[2].tf[2,3]],
    'k'
)

In [8]:
bottom_joints = []
for node in bplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                bottom_joints.append(subnode)
                
top_joints = []
for node in tplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                top_joints.append(subnode)   
                
stew_links = []
for bj, tj in zip([0,1,2,3,4,5],[2,5,0,3,4,1]):
    start = top_joints[tj].tf[0:3,3]
    end = bottom_joints[bj].tf[0:3,3]
    
    comb = np.vstack((start,end)).T
    
    stew_link, = assm.ax.plot(*comb, 'r')
    stew_links.append(stew_link)

Create functions to perform the forward kinematics

In [9]:
import modern_robotics as mr

def FK(Mlist, Slist, thetalist):
    
    # find the transforms of the link ends at each point
    Tlist = []
    chain = np.eye(4)
    for linkM, screw, theta in zip(Mlist, Slist, thetalist):
        
        # get pose of link at theta
        se3mat = mr.VecTose3(screw)
        chain = chain @ mr.MatrixExp6(se3mat*theta)
        linkT = chain @ linkM
        Tlist.append(linkT.copy())
        
    return Tlist

def update(links, Tlist):
    
    for link, T in zip(links, Tlist):
        link.tf = T @ link.relpose
        link.update()
        
    wires._verts3d = (
        np.array([Tlist[1][0,3], ArmMs[0].tf[0,3], ArmMs[1].tf[0,3], ArmMs[2].tf[0,3], ArmMs[3].tf[0,3]]),
        np.array([Tlist[1][1,3], ArmMs[0].tf[1,3], ArmMs[1].tf[1,3], ArmMs[2].tf[1,3], ArmMs[3].tf[1,3]]),
        np.array([Tlist[1][2,3], ArmMs[0].tf[2,3], ArmMs[1].tf[2,3], ArmMs[2].tf[2,3], ArmMs[3].tf[2,3]])
    )
    
    # TODO numba-fy this!
    
    # Find the whisker position

    # set arm_joint as origin
    x_1, y_1 = 0, 0
    # get relative location of crown connection in plane
    x_2, y_2 = np.sqrt(np.sum((ArmMs[0].tf[0:2,3]-Tlist[1][0:2,3])**2)), ArmMs[0].tf[2,3]-Tlist[1][2,3]
    # get relative location of whisker joint in plane
    x_3, y_3 = np.sqrt(np.sum((whisk_joint.tf[0:2,3]-Tlist[1][0:2,3])**2)), whisk_joint.tf[2,3]-Tlist[1][2,3]

    # get slope of wire
    m = (y_2-y_1)/(x_2-x_1)

    # find the intersection point on the bottom side of whisker
    ip = collision.tf[0:3,3] - collision.tf[0:3,2]*0.195 + collision.tf[0:3,1]*46.231/2

    # find radius of circle
    whisk_chord = ip - whisk_joint.tf[0:3,3]
    r = np.sqrt(np.sum(whisk_chord**2))

    # solve intersection of circle and line (and take solution with greater z)
    a = 1 + m**2
    b = -2*x_3-2*y_3*m
    c = x_3**2 + y_3**2 - r**2

    # find the coincident points in plane
    x_c = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    y_c = m*x_c

    # find the length of the wire to the coincident point
    wire_len = np.sqrt(x_c**2 + y_c**2)

    # find the 3D intersection point by tracing the wire
    wire_vec = ArmMs[0].tf[0:3,3] - Tlist[1][0:3,3]
    wire_unit = wire_vec / np.linalg.norm(wire_vec)
    p = Tlist[1][0:3,3] + wire_unit * wire_len
    
    angled_in = np.sqrt(np.sum(ForearmMs[1].tf[0:2,3])**2) < np.sqrt(np.sum(whisk_joint.tf[0:2,3])**2)
    below_wire = ForearmMs[1].tf[2,3] < p[2]
        
    if angled_in and below_wire:
        # find the angle needed to intersect this point
        R = np.eye(3)
        y_vec = p - whisk_joint.tf[0:3,3]
        R[:,1] = y_vec / np.linalg.norm(y_vec)
        R[:,2] = np.array([[1,0,0],[0, 0,-1],[0, 1,0]]) @ R[:,1] 
        R[:,0] = np.cross(R[:,1], R[:,2])
        
        # set the joint rotation to this angle
        T = whisk_joint.tf.copy()
        T[0:3,0:3] = R
        
        # set the pose of the link
        rel_pose = whisk_joint.inv_tf @ whisker.tf

        newT = T @ rel_pose
        whisker.tf = newT.copy()
        whisker.update()
        
    # Find the antenna position
    
    angled_in = np.sqrt(np.sum(ForearmMs[0].tf[0:2,3])**2) < np.sqrt(np.sum(ant_joint.tf[0:2,3])**2)
    below_whisker = ForearmMs[0].tf[2,3] < ForearmMs[1].tf[2,3]
    
    if angled_in and below_whisker:
        # find the top face of the whisker
        p = collision.tf[0:3,3] + collision.tf[0:3,2]*0.195 + collision.tf[0:3,1]*46.231/2
        
        # find the angle needed to intersect this point
        R = np.eye(3)
        y_vec = p - ant_joint.tf[0:3,3]
        R[:,1] = y_vec / np.linalg.norm(y_vec)
        R[:,2] = np.array([[1,0,0],[0, 0,-1],[0, 1,0]]) @ R[0:3,1] 
        R[:,0] = np.cross(R[0:3,1], R[0:3,2])
        
        # set the joint rotation to this angle
        T = ant_joint.tf.copy()
        T[0:3,0:3] = R
        
        # set the pose of the link
        rel_pose = ant_joint.inv_tf @ antenna.tf

        newT = T @ rel_pose
        antenna.tf = newT.copy()
        antenna.update()
        
    # Find the wire connections
    
    verts = []
    verts.append(Tlist[2][0:3,3])
    
    # is ant attached?
    
    e1 = ant_M.tf[0:3,1]
    p1 = ant_M.tf[0:3,3]
    
#     line1, = assm.ax.plot(*np.array([p1 + e1*1000, p1 - e1*1000]).T, 'r')

#     wpt, = assm.ax.plot(*wire_pt.tf[0:3,3], 'ko')
    
    p2 = wire_pt.tf[0:3,3]
    e2 = wire_pt.tf[0:3,3] - Tlist[2][0:3,3]
    e2 /= np.linalg.norm(e2)
    
#     line2, = assm.ax.plot(*np.array([p2 + e2*1000, p2 - e2*1000]).T, 'b')

    n = np.cross(e1,e2)
    n2 = np.cross(n,e2)
    c1 = p1 + (np.dot(p2-p1, n2)/np.dot(e1, n2)) * e1
    
#     cpt, = assm.ax.plot(*c1, 'ko')
    
    ant_attached = ant_M.tf[2,3] > c1[2]
    
    if ant_attached:
        verts.append(ant_M.tf[0:3,3])
    
        # is whisk attached?

        e3 = whisk_M.tf[0:3,1]
        p3 = whisk_M.tf[0:3,3]

        p4 = wire_pt.tf[0:3,3]
        e4 = ant_M.tf[0:3,3] - wire_pt.tf[0:3,3]
        e4 /= np.linalg.norm(e4)

        n3 = np.cross(e3,e4)
        n4 = np.cross(n3,e4)
        c2 = p3 + (np.dot(p4-p3, n4)/np.dot(e3, n4)) * e3

        whisk_attached = whisk_M.tf[2,3] > c2[2]
    
        if whisk_attached:
            verts.append(whisk_M.tf[0:3,3])
            
    verts.append(wire_pt.tf[0:3,3])
    verts.append(ForearmMs[2].tf[0:3,3])
    
    verts = tuple(np.array(verts).T)
    wires2._verts3d = verts
    
    update_links()

In [10]:
def update_links():
    for i, (bj, tj) in enumerate(zip([0,1,2,3,4,5],[2,5,0,3,4,1])):
        start = top_joints[tj].tf[0:3,3]
        end = bottom_joints[bj].tf[0:3,3]

        comb = np.vstack((start,end)).T

        stew_links[i]._verts3d = (comb[0], comb[1], comb[2])

def update_stewx(change):
    dx =change.new-change.old
    bplate.translate([dx,0,0])
    update_links()
    
def update_stewy(change):
    dy =change.new-change.old
    bplate.translate([0,dy,0])
    update_links()
    
def update_stewz(change):
    dz =change.new-change.old
    bplate.translate([0,0,dz])
    update_links()
    
def update_rotx(change):
    dx =change.new-change.old
    bplate.rotate_from_euler('x', dx, degrees=True)
    update_links()
    
def update_roty(change):
    dy =change.new-change.old
    bplate.rotate_from_euler('y', dy, degrees=True)
    update_links()
    
def update_rotz(change):
    dz =change.new-change.old
    bplate.rotate_from_euler('z', dz, degrees=True)
    update_links()

Create sliders!

In [11]:
import ipywidgets as widgets
theta1, theta2, theta3, = [widgets.FloatSlider(
    value=0,
    min=-np.pi/2,
    max=np.pi/2,
    step=np.pi/180,
    description=f'{desc}:',
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
) for desc in ['Theta1', 'Theta2', 'Theta3']]

theta4 = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Theta4',
    oreintation='horizontal',
    readout=True,
    readout_format = '.1f'
)

stewx, stewy, stewz, rotx, roty, rotz = [widgets.FloatSlider(
    value=0,
    min=-10.0,
    max=10.0,
    step=0.1,
    description=f'{desc}:',
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
) for desc in ['StewX', 'StewY', 'StewZ', 'RotX', 'RotY', 'RotZ']]

In [12]:
widgets.VBox([assm.plot(new=False), theta1, theta2, theta3, theta4, stewx, stewy, stewz, rotx, roty, rotz])

In [13]:
def callback(change):
    update(links, FK(Mlist, Slist, [theta1.value, theta2.value, theta3.value, theta4.value]))    

In [14]:
theta1.observe(callback, names='value')
theta2.observe(callback, names='value')
theta3.observe(callback, names='value')
theta4.observe(callback, names='value')

In [15]:
stewx.observe(update_stewx, names='value')
stewy.observe(update_stewy, names='value')
stewz.observe(update_stewz, names='value')
rotx.observe(update_rotx, names='value')
roty.observe(update_roty, names='value')
rotz.observe(update_rotz, names='value')